# Aula 24 - EXPORT DATA FRAME

In [2]:
#seção imports
import findspark
import os
import pyspark.sql.functions as R

In [3]:
from pyspark.sql import SparkSession

In [4]:
# Este seção cria o APPNAME e configura o formato de saida da função show()
# SparkSession é o processo que controla as aplicações em execução no nodes 
spark = ( 
    SparkSession.builder
    .master("local")
    .appName('ralves_sess_dframes')
    .config ("spark.sql.repl.eagerEval.enabled",True)
    .getOrCreate()
)

In [5]:
spark

In [16]:
#comando para parar a seção do spark
#spark.stop()

In [7]:
#describe python version
from platform import python_version
print(python_version())

3.7.0


In [8]:
cliente_eng = spark.createDataFrame([(1,"Mike","carro"),(1,"Mike","carro"),(1,"Mike","brinquedo"),(2,"Steve","brinquedo")],["id","cliente","produto"])

In [9]:
cliente_eng

id,cliente,produto
1,Mike,carro
1,Mike,carro
1,Mike,brinquedo
2,Steve,brinquedo


In [10]:
cliente_unico=cliente_eng.dropDuplicates().show()

+---+-------+---------+
| id|cliente|  produto|
+---+-------+---------+
|  1|   Mike|brinquedo|
|  2|  Steve|brinquedo|
|  1|   Mike|    carro|
+---+-------+---------+



In [11]:
cliente_unico=cliente_eng.dropDuplicates(['cliente']).show()

+---+-------+---------+
| id|cliente|  produto|
+---+-------+---------+
|  2|  Steve|brinquedo|
|  1|   Mike|    carro|
+---+-------+---------+



In [12]:
produto_unico=cliente_eng.dropDuplicates(['cliente','produto'])

In [17]:
#pivot table

prd_cliente= (
    produto_unico
    .withColumn('mes',R.current_date())
    .groupBy('produto')
    .pivot('cliente')
    .agg(R.count('*'))
).explain()

== Physical Plan ==
HashAggregate(keys=[produto#2], functions=[pivotfirst(cliente#1, count(1)#206L, Mike, Steve, 0, 0)])
+- Exchange hashpartitioning(produto#2, 200)
   +- HashAggregate(keys=[produto#2], functions=[partial_pivotfirst(cliente#1, count(1)#206L, Mike, Steve, 0, 0)])
      +- *(2) HashAggregate(keys=[produto#2, cliente#1], functions=[count(1)])
         +- *(2) HashAggregate(keys=[produto#2, cliente#1], functions=[partial_count(1)])
            +- *(2) HashAggregate(keys=[cliente#1, produto#2], functions=[])
               +- Exchange hashpartitioning(cliente#1, produto#2, 200)
                  +- *(1) HashAggregate(keys=[cliente#1, produto#2], functions=[])
                     +- *(1) Project [cliente#1, produto#2]
                        +- Scan ExistingRDD[id#0L,cliente#1,produto#2]


##### prd_cliente